# HTTP

In [ ]:
# day 35

import requests

api_key = 'e151956884a8aad1a198c393993f9b1a'
city = 'London'
endpoint = 'http://api.openweathermap.org/data/2.5/onecall'

parameters = {
    'lat': 22.380341,
    'lon': 114.141296,
    'appid': api_key,
    'exclude': 'current,minutely,daily'
}

response = requests.get(url = endpoint, params = parameters)
response.raise_for_status()
print(response.status_code)

data = response.json()
data

In [ ]:
weather_slice = data['hourly']

will_rain = False

for hour in weather_slice:
    condition = hour['weather'][0]['id']
    if condition < 700:
        will_rain = True
        
    if will_rain:
        print('Bring umbrella')

In [ ]:
# day 36

import requests
import datetime as dt

STOCK = "TSLA"
COMPANY_NAME = "Tesla Inc"
key = 'ZN8XKDN7Z4JX2GP2'

## STEP 1: Use https://www.alphavantage.co
# When STOCK price increase/decreases by 5% between yesterday and the day before yesterday then print("Get News").

url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={STOCK}&apikey={key}'
r = requests.get(url)
r.raise_for_status
print(r.status_code)
data = r.json()

yesterday = dt.datetime.now().date() - dt.timedelta(days = 1)
before_yesterday = yesterday - dt.timedelta(days = 1)
yesterday_price = float(data['Time Series (Daily)'][f'{yesterday}']['4. close'])
before_yesterday_price = float(data['Time Series (Daily)'][f'{before_yesterday}']['4. close'])

key2 = '1342fe0ad34a41538d77eb7bf4abdec0'
url2 = f'https://newsapi.org/v2/everything?q=tesla&from={str(before_yesterday)}&sortBy=publishedAt&apiKey={key2}'
r2 = requests.get(url2)
data2 = r2.json()
    
## STEP 2: Use https://newsapi.org
# Instead of printing ("Get News"), actually get the first 3 news pieces for the COMPANY_NAME. 

volatility = abs(yesterday_price - before_yesterday_price)/yesterday_price
news = data2['articles'][:3]
if volatility >= 0.05:
    for articles in news:
        print(news['content'])

In [ ]:
## STEP 3: Use https://www.twilio.com
# Send a seperate message with the percentage change and each article's title and description to your phone number. 
headline = [news['title'] for headline in news]
brief = [news['description'] for brief in news]

#Optional: Format the SMS message like this: 
"""
TSLA: 🔺2%
Headline: Were Hedge Funds Right About Piling Into Tesla Inc. (TSLA)?. 
Brief: We at Insider Monkey have gone over 821 13F filings that hedge funds and prominent investors are required to file by the SEC The 13F filings show the funds' and investors' portfolio positions as of March 31st, near the height of the coronavirus market crash.
or
"TSLA: 🔻5%
Headline: Were Hedge Funds Right About Piling Into Tesla Inc. (TSLA)?. 
Brief: We at Insider Monkey have gone over 821 13F filings that hedge funds and prominent investors are required to file by the SEC The 13F filings show the funds' and investors' portfolio positions as of March 31st, near the height of the coronavirus market crash.
"""


In [ ]:
# day 37

pixela_endpoint = 'https://pixe.la/v1/users'

user_para = {}

r = requests.post(url = pixela_endpoint, json = user_para)
r.raise_for_status()
data = r.json()

graph_endpoint = ''
graph_para = {}
headers = {'token': token}

graph_response = requests.post(url = graph_endpoint, json = graph_para, headers = headers)

In [ ]:
requests.put(url = endpoint, json = parameters, headers = headers)

In [ ]:
# day 38

import requests

APP_ID = '4ee99a63'
API_KEY = 'e98bf288ee365492d6368d25b3653c63'
endpoint = 'https://trackapi.nutritionix.com/v2/natural/exercise'

entry = input('Tell me which exercises you did: ')

headers = {
    'x-app-id': APP_ID,
    'x-app-key': API_KEY,
}

parameters = {
    'query': entry,
    'gender': 'male',
    'weight_kg': 75,
    'height_cm': 180,
    'age': 27,
}

r = requests.post(endpoint, json = parameters, headers = headers)
print(r.raise_for_status(), r.status_code)

In [ ]:
import datetime as dt

endpoint = 'https://api.sheety.co/phill/workout/Workout'
today = dt.datetime.now()
date = today.strftime('%d/%m/%Y')
time = today.strftime('%H:%M:%S')

def add_row(exercise, duration, calories):

    info = {
        'Workout': {
            'Date': date,
            'Time': time,
            'Exercise': exercise,
            'Duration': duration,
            'Calories': calories,
        }
    }
    
    r = requests.post(endpoint, json = info, headers = headers)

for exercise in data['exercises']:
    add_row(exercise['exercise'], exercise['duration'], exercise['calories'])

# BeautifulSoup/HTTP Parser

In [ ]:
# day 45

from bs4 import BeautifulSoup
import lxml

with open ('website.html', 'rb') as f:
    content = f.read()

# can use lxml or html.parser to parse html contents
soup = BeautifulSoup(content, 'lxml')
print(soup.title)
print(soup.title.name)
print(soup.title.string)

In [ ]:
# tidy up printed text

print(soup.prettify())

In [ ]:
all_anchor_tags = soup.find_all(name = 'a')

for tag in all_anchor_tags:
    print(tag.getText())

In [ ]:
soup.find(name = 'h1', id = 'name')
soup.find(name = 'h3', class_ = 'heading').string

In [ ]:
soup.select_one(selector = '#name')
soup.select_one(selector = '.heading')

In [ ]:
import requests

r = requests.get(url = 'https://news.ycombinator.com/')
print(r.status_code)
r.raise_for_status()

In [ ]:
webpage = r.text
soup = BeautifulSoup(webpage, 'html.parser')
articles = soup.find_all(name = 'a', class_ = 'storylink')

article_text = []
article_link = []
for article_tag in articles:
    text = article_tag.getText()
    article_text.append(text)
    link = article_tag.get('href')
    article_link.append(link)

article_upvote = [int(score.getText().replace(' points', '')) for score in soup.find_all(name = 'span', class_ = 'score')]

In [ ]:
movies = soup.find_all(name = 'h3', class_ = 'jsx-4245974604')
movies_list = [movie.getText() for movie in reversed(movies)]

with open('movies.txt', 'wb') as f:
    for movie in movies_list:
        f.write(f'{movie}\n')

# Webdriver

In [ ]:
# day 48
from selenium import webdriver

chrome_driver_path = '/Users/ching/chromedriver'
driver = webdriver.Chrome(executable_path = chrome_driver_path)
url = 'https://www.python.org/'
driver.get(url)
#price = driver.find_element_by_id('priceblock_ourprice')
#name = driver.find_element_by_name('search_query')
#item = driver.find_element_by_class_name('python-logo')
#documentation_link = driver.find_element_by_css_selector('.tier-1 element-7 a')
#link = driver.find_element_by_xpath('//*[@id="site-map"]/div[2]/div/ul/li[3]/a')
#events = driver.find_elements_by_class_name('shrubbery')
event_time = driver.find_elements_by_css_selector('.event-widget time')
event = driver.find_elements_by_css_selector('.event-widget li a')
events = {e.text: time.text for e in event for time in event_time}
print(events)

#for time in event:
#    print(time.text)
driver.quit()
# driver.close()

In [ ]:
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome(executable_path=chrome_driver_path)
driver.get('https://en.wikipedia.org/wiki/Main_Page')
#c = driver.find_element_by_css_selector('#articlecount a')
#all_portals = driver.find_element_by_link_text('All portals')
#all_portals.click()
search = driver.find_element_by_name('search')
search.send_keys('python')
search.send_keys(Keys.ENTER)
#print(c.text)

In [ ]:
driver = webdriver.Chrome(executable_path=chrome_driver_path)
driver.get('https://secure-retreat-92358.herokuapp.com/')
fname = driver.find_element_by_name('fName')
fname.send_keys('Angela')
lname = driver.find_element_by_name('lName')
lname.send_keys('Yu')
email = driver.find_element_by_name('email')
email.send_keys('abc@gmail.com')
click = driver.find_element_by_css_selector('form button')
click.click()

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chrome_driver_path = '/Users/ching/chromedriver'
url = 'https://orteil.dashnet.org/cookieclicker/'
driver = webdriver.Chrome(executable_path=chrome_driver_path)
driver.get(url)

cookie = driver.find_element_by_id('bigCookie')

while True:
    cookie.click()

In [ ]:
# day 49
from selenium.webdriver.common.keys import Keys
from time import sleep
from selenium.common.exceptions import NoSuchElementException

path = '/Users/ching/chromedriver'
driver = webdriver.Chrome(executable_path=path)
url = 'https://www.linkedin.com/jobs/search/?f_LF=f_AL&geoId=102257491&keywords=python%20developer&location=London%2C%20England%2C%20United%20Kingdom&redirect=false&position=1&pageNum=0'
driver.get(url)

"""
# auto login
login_page = 'https://www.linkedin.com/login/en?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin'
email = driver.find_element_by_id(id = 'username')
password = driver.find_element_by_id(id = 'password')
email.send_keys('email')
password.send_keys('password')
password.send_keys(Keys.ENTER)
sleep(7)
"""

# apply for jobs

is_job = True
while is_job:
    try:
        job = driver.find_elements_by_css_selector('.div href')
        for _ in job:
            button = driver.find_element_by_class_name('.jobs-s-apply button')
            button.click()
            mobile = driver.find_element_by_class_name('ember-text-field ember-view fb-single-line-text__input')
            mobile.send_keys('mobile number')
            button = driver.find_elements_by_css_selector('.footter button')
            button.click()
        section = driver.find_element_by_css_selector('.section button')
    except NoSuchElementException:
        continue

driver.quit()

In [ ]:
# day 53

form_url = 'https://forms.gle/Tvcc3uJbeKaGqNry5'
driver = webdriver.Chrome(executable_path=path)
driver.get('https://www.zillow.com/homes/for_rent/1-_beds/?searchQueryState=%7B%22mapBounds%22%3A%7B%22west%22%3A-122.90711440039063%2C%22east%22%3A-121.95954359960938%2C%22south%22%3A37.479997952295%2C%22north%22%3A38.06941019642464%7D%2C%22isMapVisible%22%3Afalse%2C%22filterState%22%3A%7B%22price%22%3A%7B%22max%22%3A872627%7D%2C%22beds%22%3A%7B%22min%22%3A1%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22mp%22%3A%7B%22max%22%3A3000%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%7D')
page = 2

while True:
    try:
        price = driver.find_elements_by_class_name('list-card-price')
        address = driver.find_elements_by_class_name('list-card-addr')
        link = driver.find_elements_by_css_selector('.list-card-info a')
    except NoSuchElementException:
        print('No such element!')
        try:
            next_page = driver.find_element_by_xpath(f'//*[@id="grid-search-results"]/div[2]/nav/ul/li[f{page}]/a')
            page += 1
        except NoSuchElementException:
            print('No such element!')
            break

# Decorator

In [ ]:
# python decorator
from time import sleep

def delay_decorator(function):
    def wrapper_function():
        sleep(2)
        # do sth before
        function()
        function()
        # do sth after
    return wrapper_function

@delay_decorator
def say_hello():
    print('Hello')
    
@delay_decorator
def say_goodbye():
    print('Goodbye')
    
def say_greeting():
    print('How are you?')
    
decorated_function = delay_decorator(say_greeting)
decorated_function()
    
#say_hello()
#say_goodbye()